In [8]:
import csv
import sys
import os
import pandas as pd
import pyproj
import numpy as np


ModuleNotFoundError: No module named 'pandas'

In [5]:
# read csv file
data_file = './dataset/year_2022.csv'
df = pd.read_csv(data_file)

# Read 2021 file
data_file_2021 = './dataset/year_2021.csv'
df_2021 = pd.read_csv(data_file_2021)

# Concatenate 2021 and 2022 files
df = pd.concat([df, df_2021])

# Drop rows with any empty cells
df = df.dropna()
df

NameError: name 'pd' is not defined

In [12]:
def separate_geo_coordinates(geo_str):
    try:
        # Split the string by comma
        easting, northing = geo_str.split(',')
        return float(easting.strip()), float(northing.strip())
    except Exception as e:
        print(f"Error separating coordinates: {e}")
        return None, None

# Step 3: Apply the function to the GeographicLocation column to get Easting and Northing
df[['GeographicEasting', 'GeographicNorthing']] = df['GeographicLocation'].apply(lambda x: pd.Series(separate_geo_coordinates(x)))

# Step 4: Initialize the UTM to WGS84 converter (for NAD83 / UTM Zone 17N)
utm_to_latlon = pyproj.Transformer.from_crs("EPSG:26917", "EPSG:4326", always_xy=True)

# Step 5: Function to convert UTM to Latitude and Longitude
def convert_utm_to_latlon(row):
    try:
        # Use Easting and Northing for conversion
        lon, lat = utm_to_latlon.transform(row['GeographicEasting'], row['GeographicNorthing'])
        return lat, lon
    except Exception as e:
        print(f"Error converting UTM to Latitude/Longitude: {e}")
        return None, None

# Step 6: Apply the conversion function to the DataFrame
df[['Latitude', 'Longitude']] = df.apply(convert_utm_to_latlon, axis=1, result_type='expand')

# Step 7: Display the updated DataFrame with Latitude and Longitude
df = df[['Latitude', 'Longitude', 'NearestIntersectionLocation', 'FinalCallType']]
df.head()




,Latitude,Longitude,NearestIntersectionLocation,FinalCallType
1,43.408025,-80.328111,Hespeler Rd / Eagle St N / Pinebush Rd,9840.0
3,43.462495,-80.485064,Hillview St / St Vincent St,9730.0
5,43.431791,-80.438667,Weber St E / Kinzie Ave,9280.0
8,43.469383,-80.583254,Woodrow Dr / Woodrow Pl,9280.0
12,43.383349,-80.441236,Apple Ridge Dr / Forest Creek Dr,9800.0


In [13]:
from sklearn.cluster import KMeans
import pandas as pd

# Step 1: Group by NearestIntersectionLocation and count crimes
intersection_crime_counts = df.groupby('NearestIntersectionLocation').size().reset_index(name='crime_count')

# Step 2: Merge with Latitude and Longitude
intersection_crime_counts = intersection_crime_counts.merge(
    df[['NearestIntersectionLocation', 'Latitude', 'Longitude']].drop_duplicates(),
    on='NearestIntersectionLocation'
)

# Step 3: Apply K-Means Clustering
X = intersection_crime_counts[['Latitude', 'Longitude', 'crime_count']]
kmeans = KMeans(n_clusters=3, random_state=42)
intersection_crime_counts['cluster'] = kmeans.fit_predict(X)

# Step 4: Calculate Crime Rate
total_crimes = intersection_crime_counts['crime_count'].sum()
intersection_crime_counts['crime_rate'] = intersection_crime_counts['crime_count'] / total_crimes

# Step 5: Get Highest and Lowest Crime Rates
highest_crime_rate = intersection_crime_counts['crime_rate'].max()
lowest_crime_rate = intersection_crime_counts['crime_rate'].min()

# Adjusted Rating Based on Crime Rate
def rate_crime(crime_rate):
    if crime_rate == lowest_crime_rate:
        return 'Very Low'
    elif crime_rate <= (lowest_crime_rate + (highest_crime_rate - lowest_crime_rate) / 3):
        return 'Low'
    elif crime_rate <= (lowest_crime_rate + 2 * (highest_crime_rate - lowest_crime_rate) / 3):
        return 'Moderate'
    else:
        return 'High'

intersection_crime_counts['rating'] = intersection_crime_counts['crime_rate'].apply(rate_crime)

# Step 6: Get Unique Crime Counts and Rank Them
unique_crime_counts = intersection_crime_counts['crime_count'].unique()
ranked_crime_counts = sorted(unique_crime_counts, reverse=True)

# Create a DataFrame for ranked crime counts
crime_ranking_df = pd.DataFrame(ranked_crime_counts, columns=['crime_count'])
crime_ranking_df['rank'] = range(1, len(crime_ranking_df) + 1)

# Step 7: Merge Rankings with the Main DataFrame
intersection_crime_counts = intersection_crime_counts.merge(
    crime_ranking_df,
    on='crime_count',
    how='left'
)

# Display the final DataFrame with crime rate, rating, and ranking
df = intersection_crime_counts[['NearestIntersectionLocation','crime_rate', 'rating', 'Latitude', 'Longitude']]




,NearestIntersectionLocation,crime_rate,rating,Latitude,Longitude
0,AVALON PL,0.000054,Low,43.426878,-80.489885
1,Abbotsford St / Hespeler Rd / Wauchope Ave,0.000242,Low,43.380378,-80.319569
2,Abbotsford St / Sekura Cres,0.000027,Low,43.380692,-80.317772
3,Abbotswood Crt / Redfox Rd,0.000013,Very Low,43.496526,-80.516744
4,Abbott Cres / Holland Cir / Michigan Ave,0.000054,Low,43.447477,-80.309684
...,...,...,...,...,...
6744,Woolwich St S,0.000054,Low,43.464567,-80.405819
6745,York Pl / Laneway 183,0.000054,Low,43.366661,-80.313870
6746,Zeller Cres / Zeller Dr,0.000040,Low,43.452772,-80.410554
6747,Zeller Crt / Zeller Dr,0.000040,Low,43.451068,-80.422300


In [9]:
# Write the final DataFrame to a CSV file
output_file = './dataset/crime_rate.csv'
df.to_csv(output_file, index=False)


NameError: name 'df' is not defined